# Buscanto os Valores das Ações

In [43]:
from pandas_datareader import data as pdr
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go

yf.pdr_override()

In [44]:
# Especifique o símbolo da ação que você deseja buscar
portfolio = ["PETR4.SA", "VALE3.SA"]  # Apple Inc. como exemplo

initial_date = "2010-07-01"
final_date = "2023-09-07"

In [45]:
def get_stock_prices(portfolio, initial_date, final_date, save=False):
    """
    Obtém os preços de ações de um portfólio de símbolos durante um período de tempo especificado.

    Parâmetros:
    portfolio (list): Uma lista de símbolos de ações para os quais deseja obter os preços.
    initial_date (str): A data de início no formato 'YYYY-MM-DD'.
    final_date (str): A data de término no formato 'YYYY-MM-DD'.
    save (bool, nome, opcional): Escreva o nome que deseja salvar o arquivo, os dados serão salvos em um arquivo CSV com o nome especificado.
    
    Retorna:
    pandas.DataFrame: Um DataFrame contendo os preços de ações para cada símbolo do portfólio.
    
    Exemplo de uso:
    portfolio = ["AAPL", "MSFT"]
    initial_date = "2023-01-01"
    final_date = "2023-01-31"
    stock_data = get_stock_prices(portfolio, initial_date, final_date, save="stock_prices")
    """
    df_out = pd.DataFrame()
    for symbol in portfolio:

        prices = pdr.get_data_yahoo(symbol, initial_date, final_date)
        prices["Date_time"] = prices.index
        prices["Ticker"] = symbol
        prices.reset_index(drop=True, inplace=True)
        df_out = pd.concat([df_out, prices], ignore_index=True)

    df_out = df_out.rename(columns={"Volume": "Vol", "Adj Close": "Adjclose"})
    if save:
        df_out.to_csv(f"{save}.csv")
        print(f"Saved file: {save}.csv")
    return df_out

In [46]:
# Obtenha os dados da cotação
prices = get_stock_prices(portfolio, initial_date, final_date)

# Exiba a cotação
prices.head()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adjclose,Vol,Date_time,Ticker
0,27.110001,27.209999,26.309999,26.450001,7.960013,18632300,2010-07-01,PETR4.SA
1,26.719999,26.990000,26.520000,26.770000,8.056314,10088300,2010-07-02,PETR4.SA
2,26.780001,27.049999,26.540001,26.540001,7.987101,6883900,2010-07-05,PETR4.SA
3,27.000000,27.299999,26.760000,27.049999,8.140581,15896300,2010-07-06,PETR4.SA
4,27.200001,27.520000,26.780001,27.400000,8.245914,16555400,2010-07-07,PETR4.SA


In [47]:
# Conhecendo detalhes do arquivo
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6550 entries, 0 to 6549
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Open       6550 non-null   float64       
 1   High       6550 non-null   float64       
 2   Low        6550 non-null   float64       
 3   Close      6550 non-null   float64       
 4   Adjclose   6550 non-null   float64       
 5   Vol        6550 non-null   int64         
 6   Date_time  6550 non-null   datetime64[ns]
 7   Ticker     6550 non-null   object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 409.5+ KB


In [54]:
def filter_stock(df, stock):
    """
    Filtra os dados de preços de uma ação específica de um DataFrame de preços de várias ações.

    Parâmetros:
    df (pandas.DataFrame): Um DataFrame contendo os preços de várias ações, incluindo a coluna 'symbol'.
    stock (str): O símbolo da ação que você deseja filtrar.

    Retorna:
    dict: Um dicionário contendo os dados filtrados da ação, adequados para criação de gráficos de candlestick.

    Exemplo de uso:
    df = ...  # DataFrame de preços de ações
    stock_data = filter_stock(df, "AAPL")
    """
    df_temp = df[df["Ticker"]==stock]
    return {
    'x': df_temp.Date_time,
    'open': df_temp.Open,
    'close': df_temp.Close,
    'high': df_temp.High,
    'low': df_temp.Low,
    'type': 'candlestick',
    'name': stock,
    'showlegend': False
    }


In [55]:
def interactive_chart(df, stock):
    """
    Cria um gráfico interativo de candlestick para uma ação específica.

    Parâmetros:
    df (pandas.DataFrame): Um DataFrame contendo os preços de várias ações, incluindo a coluna 'symbol'.
    stock (str): O símbolo da ação que você deseja exibir no gráfico.

    Exemplo de uso:
    df = ...  # DataFrame de preços de ações
    interactive_chart(prices, "AAPL")
    """

    data = [filter_stock(df, stock)]
    layout = go.Layout()
    fig = go.Figure(data=data, layout=layout)
    fig.show()

In [56]:
interactive_chart(prices, 'PETR4.SA')